
<span type="title">模型-视图-控制器模式</span> | <span type="update">2018-09-11</span> - Version <span type="version">1.0</span>
    
    
<span type="intro"><p class="card-text">本章通过一个GUI实例介绍MVC——复合模式的王者，模型-视图-控制器模式。这种模式常被用于GUI交互场合，比如桌面APP以及JSP。MVC是多种设计模式的混合，比如组合模式、观察者模式以及策略模式。现在的类库一般对于这些模式封装较好，因此在使用JavaFX或者Swing的时候很难有深入体会。本章主要介绍经典的MVC设计模式。</p></span>

# MVC 流程

MVC 的流程如下：

![](mvc1.png)

首先，我们与之交互的是视图 View，这个对象通常不具备任何动态能力，但是其提供了大量的可供修改的方法，以备别人对其界面进行更改。视图广泛使用了组合设计模式，各种组件互相嵌套，组成了GUI的界面。

当我们点击鼠标或者按下键盘，视图将请求信息发送到控制器，一般而言，在视图中实现对于事件的监听，但是不进行处理。

控制器使用策略设计模式，用来对传递得到的事件进行处理，其和视图耦合紧密。一般而言，我们不能对每个事件都附送视图的所有组件以备控制器更改，所以，一般而言，我们在视图中提供预先的内置的更改和重绘界面的方法，但是不调用。

控制器的策略模式就是这样，视图通过传递事件，控制器内保存视图的一个实例，用来调用其方法以重绘界面。控制器控制着调用何种方法进行绘制。

分离控制器和视图分离了责任，是良好的OOP设计实践。

当控制器得到事件时，其一般会从视图实例中找到对应的方法来重绘视图，比如提醒“正在处理”。之后，调用模型中的方法来改变其状态，以响应事件。模型定义了所有的程序逻辑，在模型中进行处理后如何获得返回值呢？模型一般会实现观察者模式，二控制器和视图则用来注册观察对象，这样的话，模型驱动视图和控制器进行变换，如果直接和视图交互，则直接重绘，反之，通过控制器进一步处理后重绘视图。

模型通知视图已经改变，以及视图获取其状态信息这两步通过观察者模式来进行处理，比如模型返回一个int，而视图则显示在label中，视图实现update方法，由模型调用这个接口来直接返回。

# 实例代码

## 视图

```java
package com.mazhangjing.beat;

import javafx.application.Application;
import javafx.application.Platform;
import javafx.concurrent.Task;
import javafx.event.*;
import javafx.geometry.Insets;
import javafx.scene.*;
import javafx.scene.control.*;
import javafx.scene.layout.*;
import javafx.scene.paint.Color;
import javafx.scene.paint.Paint;
import javafx.scene.text.Font;
import javafx.stage.Stage;

import java.util.Iterator;

public class BeatView extends Application implements BeatListener {
    Stage stage;
    public void update(Integer beat) { return; }
    public void update(String info) {
        this.drawInformation(info);
    }

    BeatModel model;
    BeatController controller;
    public BeatView() {
        this.model = new BeatModel(200);
        this.controller = new BeatController(model,this);
        //this.model.registerListener(this);
    }
    Button reduce = new Button("<<");
    Button add = new Button(">>");
    Button set = new Button("Set");
    Button start = new Button("Start");
    Button end = new Button("End");
    TextField number = new TextField();
    ProgressBar bar = new ProgressBar();
    Label label = new Label();
    ColorPicker picker = new ColorPicker();

    public static void main(String[] args) {
        launch(args);
    }

    Parent initScreen(Stage stage) {
        this.stage = stage;
        VBox root = new VBox(); root.setStyle("-fx-alignment: center;-fx-spacing: 40");
        HBox group = new HBox(); group.setStyle("-fx-alignment: center;-fx-spacing: 10");
        HBox cgroup = new HBox(); cgroup.setStyle("-fx-alignment: center;-fx-spacing: 10");
        HBox controlGroup = new HBox(); controlGroup.setStyle("-fx-alignment: center;-fx-spacing: 10;-fx-padding: 20");

        number.setPromptText("Input beat (integer)");
        bar.setMinSize(280,60.0);
        bar.setProgress(0.0);
        number.setText("200");
        label.setFont(Font.font(15));

        group.getChildren().addAll(number,set,reduce,add);
        root.getChildren().addAll(bar,group,controlGroup,cgroup,label);
        controlGroup.getChildren().addAll(start,end);
        cgroup.getChildren().addAll(new Label("Button Color:"),picker);

        setButtonsDisable(true,true,true,true,false,true);
        setUpListener();

        return root;
    }

    void setButtonsDisable(Boolean state0,
            Boolean state1,Boolean state2,Boolean state3, Boolean state4, Boolean state5) {
        number.setDisable(state0);
        set.setDisable(state1);
        add.setDisable(state2);
        reduce.setDisable(state3);
        start.setDisable(state4);
        end.setDisable(state5);
    }

    void setUpListener() {
        set.setOnAction((event -> {
            controller.setBeats(Integer.parseInt(number.getText().trim()));
        }));
        add.setOnAction((event -> {
            controller.increase();
        }));
        reduce.setOnAction((event -> {
            controller.decrease();
        }));
        start.setOnAction(event -> {
            controller.start();
        });
        end.setOnAction(event -> {
            controller.end();
        });
        picker.setOnAction(event -> {
            if (picker.getValue() != null) {
                Background background = new Background(new BackgroundFill(
                        picker.getValue(),new CornerRadii(
                        (double)3
                ),new Insets(1,1,1,1)
                ));
                add.setBackground(background);
                reduce.setBackground(background);
                start.setBackground(background);
                end.setBackground(background);
                set.setBackground(background);
            }
        });
    }

    void setProgress(Integer beat) {
        bar.setProgress(beat/100.0);
    }

    void setNumber(Integer num) {
        number.setText(num.toString());
    }

    void drawInformation(String info) {
        label.setVisible(true);
        label.setText(info);
    }

    @Override
    public void start(Stage primaryStage) {
        Parent root = initScreen(primaryStage);
        primaryStage.setTitle("View");
        primaryStage.setScene(new Scene(root,400,400));
        primaryStage.show();
    }
}
```

可以看到，除了初始化静态界面以及绑定监听器以外，我们提供了很多方法用于重绘界面。但是却没有调用的代码。通过创建一个控制器和模型来实现策略和监听。注意我们注册了自身为一个监听者。

## 控制器

```java
package com.mazhangjing.beat;

import javafx.application.Platform;
import java.util.concurrent.TimeUnit;

public class BeatController implements BeatListener {
    public void update(String info) { }
    public void update(Integer beat) {
        view.setProgress(beat);
    }

    BeatModel model;
    BeatView view;

    public BeatController(BeatModel model,BeatView view) {
        this.model = model; this.view = view;
        this.model.registerListener(this);
    }
    void start() {
        view.setButtonsDisable(false,false,false,false,true,false);
        view.drawInformation("Beat App is Running...");
        model.start();
    }
    void end() {
        view.setButtonsDisable(true,true,true,true,false,true);
        model.stop();
        view.drawInformation("Beat App is Stopped");
    }
    void increase() {
        model.setBeat(model.getBeat()+50);
        view.setNumber(model.getBeat());
    }
    void decrease() {
        model.setBeat(model.getBeat()-50);
        view.setNumber(model.getBeat());
    }
    void setBeats(Integer beats) {
        model.setBeat(beats);
    }
}
```

这个控制器主要用于处理视图发送的事件请求，控制器将这些事件交给模型处理，同时添加一些逻辑，比如重绘提醒信息，设置按键状态和显示状态。注意，因为我们不能直接访问视图的组件，因此需要在视图中实现准备好方法，来调用方法。或者将组件声明为public其实也可以，但是对于有规律的，可服用的代码，使用方法调用逻辑更清晰。如果全部都是对组件实例进行操作，略显麻烦。

## 模型

```java
package com.mazhangjing.beat;


import javafx.concurrent.Task;

import java.util.ArrayList;
import java.util.Iterator;
import java.util.Random;
import java.util.concurrent.TimeUnit;

public class BeatModel implements BeatObservable {
    private Integer beat;
    private Thread thread;
    private Boolean isRun;
    Random random = new Random();
    private ArrayList<BeatListener> listener = new ArrayList<>();
    public BeatModel(Integer beat) { this.beat = beat; }
    public void setBeat(Integer beat) {
        this.beat = beat;
    }
    public Integer getBeat() {
        return this.beat;
    }
    public void start() {
        thread = new Thread(
                new Runnable() {
                    public void run() {
                        while (isRun) {
                            try {Thread.sleep(beat);} catch (Exception e) {}
                            notifyListeners(random.nextInt(100));
                        }
                        return;
                    }
                }
        );
        isRun = true;
        thread.setDaemon(true);
        thread.start();
    }


    public void stop() {
        isRun = false;
    }
    public void registerListener(BeatListener newListener) {
        if (listener.contains(newListener)) return;
        else listener.add(newListener);
    }
    public void removeListener(BeatListener oldListener) {
        if (listener.contains(oldListener)) listener.remove(oldListener);
        else return;
    }
    public void notifyListeners(Integer beat) {
        Iterator<BeatListener> iterator = listener.iterator();
        while (iterator.hasNext()) { iterator.next().update(beat); }
    }
    public void notifyListeners(String info) {
        Iterator<BeatListener> iterator = listener.iterator();
        while (iterator.hasNext()) { iterator.next().update(info); }
    }
}

```

模型提供了程序所有的逻辑，比如开始，停止，更新观察者，改变值等。我们在这里使用了一个线程，间隔一段时间调用通知API来通知观察者，而观察者自身实现了upadte方法并且已经注册，因此就会被直接调用，这里直接向视图更新信息，用来重绘指示器。此外，可以向控制器更新信息，可以看到 notifyListeners 方法有两个，分别对不同组件和部分进行状态更新。

这种方法很棒，模型只用处理一个无限循环，然后调用“通知”API，它甚至不知道这个通知的调用到底做了什么！模型唯一知道的就是它的方法被调用了，它需要做些什么，它完全不清楚外界的情况，这种解耦非常好。

对于视图，它也一无所知，只知道有一个值被频繁的传递回来，而不知道是谁传递的，它只是调用自己写好的方法来重绘页面，但是它并不决定如何重绘。

对于控制器而言，它和视图耦合紧密，其决定视图展示的逻辑，比如按下开始按钮则屏蔽它，按下停止则弹起开始按钮，屏蔽其余所有组件。控制器决定视图的动态展示。同时对于模型进行调用，但是它也不是完全明白，模型干了什么，它只是知道要调用模型的某些方法，模型给它通知，有时候甚至连通知都不给，直接更新视图。偶尔有时候会返回指令，然后控制器根据此重绘视图。控制器在总体上也不了解模型是做什么的。

# MVC 的变体

在JSP中，MVC通常采用这样的流程：

![](mvc2.png)

可以看到，现在可以随时换掉控制器（请求别的URL以使用别的Servlet提供服务）完成行为响应，此外，我们不再强调视图内部组件的组合，可以由控制器直接返回新的页面视图。模型也不再和视图进行交互（通知其重绘界面），而是和控制器交互，因为它们都是Java对象，而视图则是HTML/Javascript。而由控制器更新JSP视图。

此外，MVC和之前的一样。

当然，这只是经典的Java EE Web 设计，当前的Websocket以及Ajax提供了丰富的异步能力，可以直接在视图进行重绘。